In [ ]:
from pathlib import Path
from itertools import chain
import os
import shutil

In [ ]:
import pandas as pd

In [ ]:
from dautil.util import map_parallel

In [ ]:
def md5(fname):
    '''https://stackoverflow.com/a/3431838'''
    import hashlib

    hash_md5 = hashlib.md5()
    with open(fname, "rb") as f:
        for chunk in iter(lambda: f.read(4096), b""):
            hash_md5.update(chunk)
    return hash_md5.hexdigest()

In [ ]:
def glob(path):
    return list(chain(
    path.glob('**/*.epub'),
    path.glob('**/*.PDF'),
    path.glob('**/*.webarchive'),
    path.glob('**/*.pdf'),
))

In [ ]:
# export from Papers 3 "PDF Files and Media", without annotation
path_original = Path('~/Downloads/temp-papers-export-original').expanduser()
# export from Papers 3 "PDF Files and Media", with annotation
path_annotated = Path('~/Downloads/temp-papers-export').expanduser()
# ZotFile location
path_zot = Path('~/iCloud/ZotFile').expanduser()

# Source—original

In [ ]:
paths = glob(path_original)

In [ ]:
%time md5sums = map_parallel(md5, paths, mode='multithreading', processes=os.cpu_count())

In [ ]:
df = pd.DataFrame(
    [
        [path.relative_to(path_original) for path in paths],
        md5sums
    ],
    index=['path', 'md5']
).T

In [ ]:
# per md5sum collided file, only need 1
df = df.groupby('md5').first()

In [ ]:
df

In [ ]:
df.loc['9ab299ca0b68b5966416b734f6defaa3']

# ZotFile

In [ ]:
paths = glob(path_zot)

In [ ]:
%time md5sums = map_parallel(md5, paths, mode='multithreading', processes=os.cpu_count())

In [ ]:
df_zot = pd.DataFrame(
    [
        paths,
        md5sums
    ],
    index=['path', 'md5']
).T

In [ ]:
assert (df_zot.md5.value_counts() > 1).sum() == 0

In [ ]:
df_zot = df_zot.merge(df, how='left', left_on='md5', right_index=True, suffixes=('', '_source'))

In [ ]:
df_zot

In [ ]:
assert df_zot.path_source.isna().sum() == 0

In [ ]:
%%time
for name, row in df_zot.iterrows():
    shutil.copy(path_annotated / row.path_source, row.path)